In [1]:
import jamspell
import sqlite3 as sqlite
import tqdm
import os
import difflib

In [2]:
conn = sqlite.connect('../../data/jokes.db')
cursor = conn.cursor()

In [3]:
corrector = jamspell.TSpellCorrector()
corrector.LoadLangModel('../../data/ru_small.bin')

True

In [12]:
a = cursor.execute("""SELECT COUNT(*) FROM jokes j
WHERE j.likes == "..."
GROUP BY j.likes""")

In [14]:
next(a)

StopIteration: 

In [8]:
for row in tqdm.tqdm(cursor.execute("SELECT * FROM jokes LIMIT 3")):
    print(row[1], '\n')
    print(corrector.FixFragment(row[1]), '\n')

3it [00:00, 38.98it/s]

Про самообслуживание подростков. Когда к нашей соседке по комнате (1 курс) приехала младшая сестра, она воскликнула утром:— Оффигеть, ты кровать сама заправляешь! 

Про самообслуживание подростков. Когда к нашей соседке по комнате (1 курс) приехала младшая сестра, она воскликнула утром:— Оффигеть, ты кровать сама заправляешь! 

Участник: Хочу напомнить админам, что права о свободе слова защищены конституцией рф и действуют на всей её территории. а также. все равны перед законом. баном за политику были нарушены мои конституционные права на свободу слова и вполне могу подать на форум в суд. так что вам стоит бояться суда не только за политику, но и за цензуру.Модератор: Я тебе уже как-то писал:право высказывать свою точку зрения не означает автоматически обязанность всех и каждого предоставлять ресурс для публикации этой точки зрения.Так что иди, судись.Бан.Админ: Перезабанил на подольше. 

Участник: Хочу напомнить динамо, что права о свободе слова защищены конституцией рф и действуют на

In [32]:
fixed = []

sql = "SELECT * FROM jokes"
num_corrected = 0
whole_num = 0

for row in tqdm.tqdm(cursor.execute(sql)):
    corrected = corrector.FixFragment(row[1])
    if corrected != row[1]:
        num_corrected += 1
    whole_num += 1
    fixed.append({
        'id': row[0],
        'text': corrected,
        'likes': row[2],
        'date': row[3]
    })

73054it [11:24, 106.78it/s]


In [33]:
print(num_corrected/whole_num)

0.6012401784980973


## Save corrected to db

In [34]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS corrected_jokes
(id int PRIMARY KEY, jokes text, likes text, date text)
""")
conn.commit()

In [36]:
cursor.executemany(
    "INSERT INTO jokes VALUES (?,?,?,?)", 
    [(joke['id'], joke['text'], joke['likes'], joke['date']) 
     for joke in fixed])

In [37]:
conn.commit()

In [38]:
cursor.execute("""
ALTER TABLE jokes ADD COLUMN corrected text
""")
conn.commit()

In [39]:
cursor.executemany(
    "INSERT INTO jokes (corrected) VALUES (?)", 
    [(joke['text'], ) for joke in fixed])

In [40]:
conn.commit()

In [45]:
a = cursor.execute("""
SELECT corrected FROM jokes LIMIT 5
""")

In [46]:
for row in a:
    print(row)

(None,)
(None,)
(None,)
(None,)
(None,)


In [47]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS data
(id int PRIMARY KEY, jokes text, corrected_jokes text, likes text, date text)
""")
conn.commit()

In [49]:
cursor.execute("""
INSERT INTO data SELECT jokes.id, jokes.jokes, corrected_jokes.jokes, jokes.likes, jokes.date FROM jokes, corrected_jokes
""")

In [51]:
conn.commit()

In [3]:
for row in cursor.execute("SELECT * FROM data LIMIT 5"):
    print(row)

In [ ]:
cursor.et